# Úkol č. 2 - předzpracování dat a binární klasifikace (do 7. prosince)

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na trénovací, testovací a případně i validační množinu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí testovací množiny. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) s predikcemi pro vyhodnocovací data.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.misc import logsumexp
import sklearn.metrics as metrics
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
pd.set_option('display.max_rows', 2000)

In [ ]:
def upravaDat(data):
    display(data[data.drop(['cabin','age','home.dest'],axis=1).isnull().any(axis=1)])
    data = fare(data)
    data = embarked(data)    
    data = pclass(data)
    data = name(data)
    data = sex(data)    
    data = sibsp(data)
    data = parch(data)
    data = ticket(data)
    data = cabin(data)
    data = age(data)
    data = home_dest(data)
    return data

In [ ]:
def pclass(data):
    print('\n-----------pclass-------\n')
    display(data['pclass'].value_counts())
    return data

In [ ]:
def name(data):
    data.insert(3,'titul',data.iloc[:,3].str.split(' ').str[1])
    for i in data.index:
        if data.loc[i,'titul'] != 'Mr.' and data.loc[i,'titul'] != 'Miss.' and data.loc[i,'titul'] != 'Mrs.' and data.loc[i,'titul'] != 'Master.':
            data.loc[i,'titul'] = 'Other'
    data = data.drop(['name'],axis=1)
    data['titul'] = data['titul'].map({'Mr.' : 0 , 'Miss.' : 1 , 'Mrs.' : 2 , 'Master.' : 3 , 'Other' : 4 })
    return data

In [ ]:
def sex(data):
    data['sex'] = data['sex'].map({'female' : 0 , 'male' : 1 })
    data['sex'] = data['sex'].astype('int64')
    return data

In [ ]:
def age(data):
    print('\n-----------age-------\n')
    display(data.corr().age)
    
    for i in np.arange(1, 4, 1):
        for j in np.arange(0, 7, 1):
            data.loc[(data['age'].isnull()) & (data['pclass'] == i) & (data['sibsp'] == j), 'age'] = data[ (data['pclass']== i) & (data['sibsp'] == j) ]['age'].mean()
    
    data['age'] = pd.cut(data['age'].astype('int64'),bins=[0,10,18,25,30,45,55,1000],labels=[0,1,2,3,4,5,6],include_lowest=True)
    data['age'] = data['age'].astype('int64')
    display(data['age'].value_counts())
    return data

In [ ]:
def sibsp(data):

    data.loc[data['sibsp'] > 4, 'sibsp'] = 5

    print('\n-----------sibsp-------\n')
    display(data['sibsp'].value_counts())
    return data

In [ ]:
def parch(data):

    data.loc[data['parch'] > 4, 'parch'] = 5
    print('\n-----------parch-------\n')
    display(data['parch'].value_counts())
    return data

In [ ]:
def ticket(data):
    data = data.drop(['ticket'],axis=1)
    return data

In [ ]:
def fare(data):
    data['fare'] = data['fare'].fillna(data[data['pclass']==3]['fare'].value_counts().keys()[0])
    data['fare'] = pd.cut(data['fare'].astype('int64'),bins=[0,10,30,70,100,1000],labels=[0,1,2,3,4],include_lowest=True)
    data['fare'] = data['fare'].astype('int64')
    return data

In [ ]:
def cabin(data):
    print('\n-----------cabin-------\n')
    data.insert(11,'paluba',data.loc[:,'cabin'].str[0])
    data.insert(11,'pozice',data.loc[:,'cabin'].str[1:].str.split(' ').str[0])
    data = data.drop(['cabin'],axis=1)
    for i in data.index:
        if data.loc[i,'pozice'] == '':
            data.loc[i,'pozice'] = np.nan
    data['pozice'] = data['pozice'].fillna(int(data[data.notnull().pozice]['pozice'].astype('int64').mean()))
    data['paluba'] = data['paluba'].fillna('C')    
    data.loc[data['paluba'] > 'F', 'paluba'] = 'G'
    display(data['paluba'].value_counts())
    data['paluba'] = data['paluba'].map({'A' : 0 , 'B' : 1 , 'C' : 2 , 'D' : 3 , 'E' : 4 , 'F' : 5 , 'G' : 6})
    
    data['pozice'] = pd.cut(data['pozice'].astype('int64'),bins=[0,20,45,50,100,1000],labels=[0,1,2,3,4],include_lowest=True)
    data['pozice'] = data['pozice'].astype('int64')
    
    display(data['pozice'].value_counts())
    
    return data

In [ ]:
def embarked(data):
    data['embarked'] = data['embarked'].fillna(data['embarked'].value_counts().keys()[0])
    print('\n-----------embarked-------\n')
    display(data['embarked'].value_counts())
    data['embarked'] = data['embarked'].map({'S' : 1 , 'C' : 0 , 'Q' : 2})
    
    return data

In [ ]:
def home_dest(data):
    data = data.drop(['home.dest'],axis=1)
    return data

In [ ]:
def forest(data):
    X, X_test, Y, Y_test = train_test_split(data.drop(columns = ['ID', 'survived']), data['survived'], test_size=0.25, random_state=42)
    Xtrain, Xval, Ytrain, Yval = train_test_split(X, Y, test_size=0.25, random_state=235)
    param_grid = {
        'n_estimators': range(1,100,5),
        'max_depth': range(1,6)
    }
    param_comb = ParameterGrid(param_grid)
    val_acc = []
    for param in param_comb:
        ret = RandomForestClassifier(**param)
        ret.fit(Xtrain, Ytrain)
        val_acc.append(math.sqrt(metrics.mean_squared_error(Yval, ret.predict(Xval))))
    best_param = param_comb[np.argmin(val_acc)]
    ret = RandomForestClassifier(**best_param)
    ret.fit(Xtrain, Ytrain)
    print('accuracy score (train): {0:.2f}'.format(metrics.accuracy_score(Y, ret.predict(X))*100),'%')
    print('accuracy score (test): {0:.2f}'.format(metrics.accuracy_score(Y_test, ret.predict(X_test))*100),'%')
    return ret

In [ ]:
data = pd.read_csv('data.csv')
data.info()
data = upravaDat(data)
data.info()


In [ ]:
evaluate = pd.read_csv('evaluation.csv')
evaluate.info()
evaluate = upravaDat(evaluate)
evaluate.info()

In [ ]:
ret = forest(data)
result = pd.DataFrame(
    {'ID': evaluate['ID'],
     'predikce_preziti': ret.predict(evaluate.drop(columns = ['ID']))
    })
result.to_csv('result.csv', index=False)